# <u>Teams</u>

AgentChat creates [**agent teams**](https://microsoft.github.io/autogen/stable//user-guide/agentchat-user-guide/tutorial/teams.html) for them to interact with each other.

Each team achieves **a goal** by working together.

**First we will**
1. Create a team
2. Run a team

While doing this we will <u>**observe team's behavior**</u>, which is crucial along with common **operations to <u>control team's behavior**</u>

**Team presets**
1. **RoundRobinGroupChat** - runs group chat with agents participating sequentially (covered here)
2. **SelectorGroupChat** - (advanced) team that selects the next participant using a judge (LLM)
3. **MagneticOneGroupChat** - (advanced) generalist multi-agent to solve open ended web and file-based tasks
4. **Swarm** - (advanced)

**When to use a team?**

First start with single agent. Use all it's resources and if it still fails to solve the problem, use a team.

## <u>RoundRobinGroupChat</u>

### <u>Creating a team</u>

All agents share the same context.

All agents take turn to answer sequentially.

Each agent broadcasts its answer to all other agents.

Example below : 2 Agents, *reflection pattern*, TextMentionTermination.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

vendor_agent = AssistantAgent(
    "vendor",
    model_client=model_client,
    system_message="You are a vegetable vendor from Pune, India. Once some vegetables are sold, reply with 'THANK YOU'",
)

buyer_agent = AssistantAgent(
    "buyer",
    model_client=model_client,
    system_message="You are vegetable buyer conversing with a vendor in subji mandi. Ask for the vegetables you want and buy them at appropriate price.",
)

text_termination = TextMentionTermination("THANK YOU")

team = RoundRobinGroupChat([vendor_agent, buyer_agent], termination_condition=text_termination)

### RUn

In [5]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)
print("\n")
print("Type of result is : ", type(result))

messages=[TextMessage(id='e2044a54-ccd2-4a8e-a435-de7d634572b7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 5, 57, 12, 504326, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='4b71d157-30f5-4079-90be-5e83419a0fbc', source='vendor', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=83), metadata={}, created_at=datetime.datetime(2025, 9, 20, 5, 57, 16, 829651, tzinfo=datetime.timezone.utc), content="In fall's embrace, the leaves do dance,  \nIn shades of amber, gold, and chance.  \nThe crisp air whispers, nature's song,  \nAs harvest tales of autumn prolong.  \n\nPumpkins glow in the twilight's hue,  \nAnd breezes carry a scent so true.  \nGather and cherish, the bounty's call,  \nIn the heart of the season, we find our all.  ", type='TextMessage'), TextMessage(id='c580662d-dd0b-47a3-a057-437cc030172a', source='buyer', models_usage=RequestUsage(prompt_tokens=1

### <u>result from teams.run</u>

We see above that teams.run gives us a TaskResult object.

Find below 5 characteristics of this object

<img src="images/image_13.png" width="600">

In [21]:
result.__dict__.keys()

dict_keys(['messages', 'stop_reason'])

In [23]:
print("STOP REASON : ", result.stop_reason)

STOP REASON :  Text 'THANK YOU' mentioned


In [24]:
print("\nTypes of result.messages contents is : \n")
[print(type(i)) for i in result.messages]


Types of result.messages contents is : 

<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>


[None, None, None, None, None, None, None, None]

In [4]:
for i in result.messages:
    print(i.source, "\n", "----------", "\n", i.content, "\n")

user 
 ---------- 
 Write a short poem about the fall season. 

vendor 
 ---------- 
 In fall's embrace, the leaves do dance,  
In shades of amber, gold, and chance.  
The crisp air whispers, nature's song,  
As harvest tales of autumn prolong.  

Pumpkins glow in the twilight's hue,  
And breezes carry a scent so true.  
Gather and cherish, the bounty's call,  
In the heart of the season, we find our all.   

buyer 
 ---------- 
 That was lovely! Now, speaking of autumn, I could use some fresh vegetables for the season. Do you have any pumpkins, sweet potatoes, or perhaps some hearty greens? What are your prices today? 

vendor 
 ---------- 
 Yes, I have fresh pumpkins available for ₹30 each, sweet potatoes for ₹40 per kg, and hearty greens like spinach and mustard greens for ₹20 per bunch. Would you like to buy any of these veggies?  

buyer 
 ---------- 
 Great prices! I’d like to take 2 pumpkins and 2 kg of sweet potatoes, please. And let's add 3 bunches of mustard greens. What wil

### <u>Observing a team</u>

Observe a team using **run_stream** method.

In [26]:
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Start the task of buying vegetables. Mention vegetables local to maharashtra"):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)
        print("\n")

id='f7d0fe42-49e1-4fa0-837f-29bf2017dbd3' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 9, 20, 6, 27, 43, 511319, tzinfo=datetime.timezone.utc) content='Start the task of buying vegetables. Mention vegetables local to maharashtra' type='TextMessage'


id='310a3e39-3014-4607-b69d-0d2e522afc4d' source='vendor' models_usage=RequestUsage(prompt_tokens=48, completion_tokens=120) metadata={} created_at=datetime.datetime(2025, 9, 20, 6, 27, 47, 468887, tzinfo=datetime.timezone.utc) content='Sure! Here are some local vegetables from Maharashtra that you might find interesting:\n\n1. Bhindi (Okra)\n2. Vangi (Brinjal/Eggplant)\n3. Batata (Potato)\n4. Kanda (Onion)\n5. Tamata (Tomato)\n6. Paatal (Turmeric)\n7. Pothole Bhaji (Drumstick greens)\n8. Kothimbir (Coriander leaves)\n9. Methi (Fenugreek)\n10. Palak (Spinach)\n\nLet me know what you would like to buy!' type='TextMessage'


id='703fb329-6549-41f6-a428-e9ee6ce75a18' source='buyer' models_usage=RequestUsage(p

#### <u>Doing the same above with Console</u>

This is the actual use of **Console** method. Whatever run stream gives, it outputs with better formatting.

In [27]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Hello. How are you?"))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Hello. How are you?
---------- TextMessage (vendor) ----------
Hello! I’m doing well, thank you. How can I assist you today with vegetables?
---------- TextMessage (buyer) ----------
I’m looking to buy some fresh vegetables for the week. Can you please show me your best quality tomatoes, onions, and spinach? What are your prices for these?
---------- TextMessage (vendor) ----------
Sure! Here are my best quality vegetables and their prices:

- **Tomatoes**: ₹40 per kg
- **Onions**: ₹30 per kg
- **Spinach**: ₹20 per bunch

Let me know how much you would like to buy!
---------- TextMessage (buyer) ----------
Great! I would like to buy 2 kg of tomatoes, 3 kg of onions, and 5 bunches of spinach. Can you give me the total price for that?
---------- TextMessage (vendor) ----------
Sure! Here’s the total for your order:

- **Tomatoes (2 kg)**: ₹80
- **Onions (3 kg)**: ₹90
- **Spinach (5 bunches)**: ₹100

**Total Price:** ₹270

Would you like to proceed

TaskResult(messages=[TextMessage(id='72b9e660-8452-4abc-b3a5-a886a8d51fdc', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 6, 29, 47, 355265, tzinfo=datetime.timezone.utc), content='Hello. How are you?', type='TextMessage'), TextMessage(id='985c217d-3dbe-4b95-819b-14051a4efc20', source='vendor', models_usage=RequestUsage(prompt_tokens=40, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 9, 20, 6, 29, 48, 436613, tzinfo=datetime.timezone.utc), content='Hello! I’m doing well, thank you. How can I assist you today with vegetables?', type='TextMessage'), TextMessage(id='9f542d42-de14-4f59-a054-cee860970fb9', source='buyer', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=34), metadata={}, created_at=datetime.datetime(2025, 9, 20, 6, 29, 49, 642606, tzinfo=datetime.timezone.utc), content='I’m looking to buy some fresh vegetables for the week. Can you please show me your best quality tomatoes, onions, and spinach? 

### <u>Resetting a team</u>

As we saw above, to reset a team we do **team.reset()**.

This will clear the team's state including all agents. It internally calls each agent's **on_reset()** method to clear agent's state.

I did some research on what exactly is this **state**

Below I found a method **save_state** which gives us a very complicated state dictionary.

Below we also hit reset, which wipes out this **state** dictionary.

In [29]:
print(dir(team))

['DEFAULT_DESCRIPTION', 'DEFAULT_NAME', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__protocol_attrs__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_base_group_chat_manager_class', '_create_group_chat_manager_factory', '_create_participant_factory', '_description', '_embedded_runtime', '_emit_team_events', '_from_config', '_from_config_past_version', '_group_chat_manager_name', '_group_chat_manager_topic_type', '_group_topic_type', '_init', '_initialized', '_is_protocol', '_is_running', '_is_runtime_protocol', '_max_turns', '_message_factory', '_name

In [37]:
team_state = await team.save_state()
print(team_state)

{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'vendor': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Hello. How are you?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Hello! I’m doing well, thank you. How can I assist you today with vegetables?', 'thought': None, 'source': 'vendor', 'type': 'AssistantMessage'}, {'content': 'I’m looking to buy some fresh vegetables for the week. Can you please show me your best quality tomatoes, onions, and spinach? What are your prices for these?', 'source': 'buyer', 'type': 'UserMessage'}, {'content': 'Sure! Here are my best quality vegetables and their prices:\n\n- **Tomatoes**: ₹40 per kg\n- **Onions**: ₹30 per kg\n- **Spinach**: ₹20 per bunch\n\nLet me know how much you would like to buy!', 'thought': None, 'source': 'vendor', 'type': 'AssistantMessage'}, {'content': 'Great! I would like to buy 2 kg of toma

C:\Users\chaitanya.gokhale\AppData\Local\Temp\ipykernel_11628\3881202841.py:1: RuntimeWarning: coroutine 'BaseGroupChat.save_state' was never awaited
  team_state = await team.save_state()


In [38]:
await team.reset()
team_state = await team.save_state()
print(team_state)

{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'vendor': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': []}}, 'message_buffer': []}, 'buyer': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': []}}, 'message_buffer': []}, 'RoundRobinGroupChatManager': {'type': 'RoundRobinManagerState', 'version': '1.0.0', 'message_thread': [], 'current_turn': 0, 'next_speaker_index': 0}}}


**We see above that resetting the team changes the team_state dict**

### <u>Stopping a team</u>

1 type of stopping conditions like **TextMentionTermination** that we used are internal conditions that stop the team based on the **state**.

We can also stop the team **externally** using **ExternalTermination**

In the below code snippet
1. asyncio.create_task - starting task in the background
2. asyncio.sleep(5) - if the task gets complete (THANK YOU received) in 5 seconds then good else the next line...
3. ..The next line *external_termination.set()* manually i.e. externally terminates the task (whichever agent was speaking, he will complete what he says and the team will stop).

What happens
1. *external_termination.set()* sets a flag (like flip a switch).
2. This flag is being watched for by the team after every agent stops speaking.

In [40]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat([vendor_agent, buyer_agent], termination_condition=external_termination | text_termination)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(5)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (vendor) ----------
In the season of fall, the leaves gently sway,  
A tapestry of colors in a grand display.  
Golden and crimson, they dance in the breeze,  
Whispers of autumn rustle through the trees.  

The air turns crisp, and the days grow short,  
Harvest time comes, a joyful report.  
Pumpkins and apples, so ripe and so sweet,  
Nature's embrace, a cozy retreat.  

As the sun dips low and the nights grow long,  
We gather around, where hearts feel strong.  
In the warmth of the season, let laughter resound,  
For in the beauty of fall, joy can be found.  
---------- TextMessage (buyer) ----------
That’s a lovely poem about fall! Now, speaking of the season, I’m looking to buy some fresh vegetables for my kitchen. Can you tell me the prices for the following items: pumpkins, apples, and any seasonal greens you have?
---------- TextMessage (vendor) ----------
Sure! Here are 

TaskResult(messages=[TextMessage(id='53aafb3d-e797-46bc-abe3-6f5ff08725fc', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 7, 16, 20, 87382, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='6c2d929c-adea-4cab-ade7-b3b1f809ccac', source='vendor', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=134), metadata={}, created_at=datetime.datetime(2025, 9, 20, 7, 16, 23, 31683, tzinfo=datetime.timezone.utc), content="In the season of fall, the leaves gently sway,  \nA tapestry of colors in a grand display.  \nGolden and crimson, they dance in the breeze,  \nWhispers of autumn rustle through the trees.  \n\nThe air turns crisp, and the days grow short,  \nHarvest time comes, a joyful report.  \nPumpkins and apples, so ripe and so sweet,  \nNature's embrace, a cozy retreat.  \n\nAs the sun dips low and the nights grow long,  \nWe gather around, where hearts feel strong.

**Note what the bot did, on detecting external termination - the team put in the THANK YOU value itself so that the process exits using TextTermination itself.**

continue from resuming the team